# Lab 5 Twitter API 

### Question 1: Where the tweets at?

In [ ]:
from geopy import geocoders
import tweepy
import csv
import json

global tweets
tweets = 0
counter = 0

def geo(location):
    g = geocoders.Nominatim()
    loc = g.geocode(location)
    return loc.latitude, loc.longitude

def WriteCSV(user, text, lat, long):
    global tweets
    f = open('tweets.csv', 'a')
    write = csv.writer(f)
    write.writerow([user, text, lat, long])
    f.close()

import json

CK = 'KEY'
CS = 'KEY'
AK = 'KEY'
AS = 'KEY'

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

class CustomStreamListener(tweepy.StreamListener):
    def on_data(self, data):
        Data = json.loads(data)
        Author = Data['user']['screen_name']
        text = Data['text']
        print(Author)
        print(text)
        print(Data['place']['full_name'])
        print()
        x,y = geo(Data['place']['full_name'])
        WriteCSV(Author, text, x,y)

while True:
    try:
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        stream.filter(locations=[-122.626, 47.113, -121.754,47.87])
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue

### Question 2: Tweets on a map.

In [1]:
import pandas

tweets = pandas.read_csv('tweets.csv', encoding = 'ISO-8859-1')
tweets.head()

User                                              Tweet  \
0       susistuart  @BrettHamil We all know it's not supposed to s...   
1  CurtisM40613460  @SportsRadioKJR @johncanzanobft @cliffavril @J...   
2        Nmartin55                       @NFCCHAMPS18 Yup. Its a zoo   
3     AliWantsFood  @DavidVonderhaar To make the older call of dut...   
4         lilith2u  @mariecountryman @KittyBhagat @renemjclarke @b...   

         lat        long  
0  47.603832 -122.330062  
1  47.603832 -122.330062  
2  47.685957 -122.192025  
3  47.313494 -122.339310  
4  47.603832 -122.330062

In [5]:
import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium, geopandas

tweets = pandas.read_csv('tweets.csv', encoding = 'ISO-8859-1')
tweets.head()

panda = pandas.DataFrame(tweets, columns=['User','Tweet','lat','long'])
#print(panda)

coords = zip(panda['long'], panda['lat'])

geometry = [shapely.geometry.Point(c) for c in coords]

geolist = geopandas.GeoSeries(geometry)

geoPoints = geopandas.GeoDataFrame(
        panda,
        geometry=geolist)

geoPoints.crs=fiona.crs.from_epsg(4236)

tweetmap = folium.Map(
                location=[47.2529, -122.4443],
                zoom_start=8,
                tiles='OpenStreetMap')

tweetpoints = folium.features.GeoJson(geoPoints.to_json()).add_to(tweetmap)

tweetmap

### Question 3 - Cluster Map

In [98]:
from folium.plugins import MarkerCluster
import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium, geopandas
from geopy import geocoders
import tweepy
import csv
import json

#attempt at user input and csv saving
keywords = input('Enter a keyword to search for: ')

def geo(location):
    g = geocoders.Nominatim()
    loc = g.geocode(location)
    return loc.latitude, loc.longitude

def ClusterCSV(user, text, lat, long):
    global tweets
    f = open('cluster.csv', 'a')
    write = csv.writer(f, delimiter = '*')
    write.writerow([user, text, lat, long])
    f.close()

CK = 'KEY'
CS = 'KEY'
AK = 'KEY'
AS = 'KEY'

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

class CustomStreamListener(tweepy.StreamListener):
    def on_data(self, data):
        Data = json.loads(data)
        Author = Data['user']['screen_name']
        text = Data['text']
        print(Author)
        print(text)
        print(Data['place']['full_name'])
        print()
        x,y = geo(Data['place']['full_name'])
        ClusterCSV(Author, text, x,y)

while True:
    try:
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        stream.filter(track=[keywords], languages=["en"])
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue

#Put a map on it!
from folium.plugins import MarkerCluster
import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium, geopandas

tweets = pandas.read_csv('tweets.csv', encoding = 'ISO-8859-1')

lat = list(tweets["lat"])
long = list(tweets["long"])
tweet = list(tweets["Tweet"])
user = list(tweets["User"])

panda = pandas.DataFrame(tweets, columns = ['User','Tweet','lat','long'])
#print(panda)

coords = zip(panda['long'], panda['lat'])

geometry = [shapely.geometry.Point(c) for c in coords]

geolist = geopandas.GeoSeries(geometry)

geoPoints = geopandas.GeoDataFrame(
        panda,
        geometry = geolist)

geoPoints.crs = fiona.crs.from_epsg(4236)

tweetclustermap = folium.Map(
                location = [47.2529, -122.4443],
                zoom_start = 8,
                tiles='OpenStreetMap')

# plot MarkerCluster for popups
mycluster = MarkerCluster()
for lt, ln, us, tw in zip(lat, long, user, tweet):
    iframe = folium.IFrame(html = 'USER: ' + str(us) + ' <br> '+ 'TWEET: ' + str(tw) , width=300, height=150)
    mycluster.add_child(folium.Marker(location = [lt, ln], popup = folium.Popup(iframe), icon = folium.Icon(icon = 'cloud', color = 'green')))

tweetclustermap.add_child(mycluster)

tweetclustermap

### Question 4 - Heatmap

In [97]:
from folium.plugins import HeatMap
import pandas

import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium, geopandas

#attempt at user input and csv saving
keywords = input('Enter a keyword here: ')

def geo(location):
    g = geocoders.Nominatim()
    loc = g.geocode(location)
    return loc.latitude, loc.longitude

def HeatCSV(user, text, lat, long):
    global tweets
    f = open('heat.csv', 'a')
    write = csv.writer(f, delimiter = '*')
    write.writerow([user, text, lat, long])
    f.close()

CK = 'KEY'
CS = 'KEY'
AK = 'KEY'
AS = 'KEY'

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

class CustomStreamListener(tweepy.StreamListener):
    def on_data(self, data):
        Data = json.loads(data)
        Author = Data['user']['screen_name']
        text = Data['text']
        print(Author)
        print(text)
        print(Data['place']['full_name'])
        print()
        x,y = geo(Data['place']['full_name'])
        HeatCSV(Author, text, x,y)

while True:
    try:
        
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        stream.filter(track=[keywords], languages=["en"])
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue

#Put a map on it!
tweets = pandas.read_csv('tweets.csv', encoding = 'ISO-8859-1')
tweets.head()

panda = pandas.DataFrame(tweets, columns = ['User','Tweet','lat','long'])
#print(panda)

coords = zip(panda['long'], panda['lat'])

geometry = [shapely.geometry.Point(c) for c in coords]

geolist = geopandas.GeoSeries(geometry)

geoPoints = geopandas.GeoDataFrame(
        panda,
        geometry = geolist)

geoPoints.crs = fiona.crs.from_epsg(4236)

tweetheatmap = folium.Map(
                location = [47, -122],
                zoom_start = 8,
                tiles = 'OpenStreetMap')

# convert to (n, 2) nd-array format for heatmap
tweetheat = tweets[['lat', 'long']].values

# plot heatmap
tweetheatmap.add_child(folium.plugins.HeatMap(tweetheat, radius=15))

#points
for index, row in tweets.iterrows():
    folium.CircleMarker([row['lat'], row['long']],
                        radius = 2.5,
                        popup = row['Tweet'],
                        fill_color = '#3db7e4',
                        fill_opacity = '0.1',
                        opacity = '0.2',
                        weight = '2'
                       ).add_to(tweetheatmap)
    
tweetheatmap